In [1]:
import matplotlib
matplotlib.use("Agg")
 
# import the necessary packages
from keras.preprocessing.image import ImageDataGenerator
from keras.optimizers import Adam
from sklearn.model_selection import train_test_split
from keras.preprocessing.image import img_to_array
from keras.utils import to_categorical
from imutils import paths
import matplotlib.pyplot as plt
import numpy as np
import argparse
import random
import cv2
import os
import sys

Using TensorFlow backend.


In [2]:
data = []
labels = []
path = 'C:\\Users\\Big data\\PycharmProjects\\ML\\model0731\\600up'
imagePaths = sorted(list(paths.list_images(path)))
random.seed(42)
random.shuffle(imagePaths)

In [ ]:
for imagePath in imagePaths:
    # load the image, pre-process it, and store it in the data list
    image = cv2.imread(imagePath)
    print(imagePath)
    image = cv2.resize(image, (80, 80))
    image = img_to_array(image)
    data.append(image)

In [4]:
for imagePath in imagePaths:
    label = imagePath.split(os.path.sep)[7]
    labels.append(label)

In [5]:
labels_copy = labels
len(labels_copy)

225529

In [6]:
from sklearn.preprocessing import LabelEncoder
labelencoder = LabelEncoder()
labels = labelencoder.fit_transform(labels)

In [7]:
np.max(labels)

141

In [8]:
data = np.array(data, dtype="float32") / 255.0

In [9]:
labels = np.array(labels)

In [10]:
labels_copy2 =labels

In [11]:
(trainX, testX, trainY, testY) = train_test_split(data,
            labels, test_size=0.2, random_state=42)

In [12]:
trainY = to_categorical(trainY, num_classes=142)
testY = to_categorical(testY, num_classes=142)

In [22]:
from keras.models import Sequential
from keras.layers.convolutional import Conv2D
from keras.layers.convolutional import MaxPooling2D
from keras.layers.core import Activation
from keras.layers.core import Flatten
from keras.layers.core import Dense
from keras.layers.core import Dropout
from keras import backend as K


In [19]:
from keras.applications.xception import Xception
from keras.layers import Input
input_tensor = Input(shape=(80,80,3))
model = Xception(include_top=True,
                weights=None,
                input_tensor=input_tensor,
                input_shape=None,
                pooling=None,
                classes=142)

In [ ]:
model.summary()

In [22]:
EPOCHS = 100
INIT_LR = 1e-3
BS = 32
opt = Adam(lr=INIT_LR, decay=INIT_LR / EPOCHS)
model.compile(loss="categorical_crossentropy", optimizer=opt,
    metrics=["accuracy"])
from keras.callbacks import EarlyStopping, CSVLogger
estop = EarlyStopping(monitor='val_loss', patience=10)
logger = CSVLogger('model_xception.log')

In [23]:
aug = ImageDataGenerator(rotation_range=30, width_shift_range=0.1,
    height_shift_range=0.1, shear_range=0.2, zoom_range=0.2,
    horizontal_flip=True, fill_mode="nearest")

In [ ]:
H = model.fit_generator(aug.flow(trainX, trainY, batch_size=BS),
    validation_data=(testX, testY), steps_per_epoch=len(trainX) // BS,
    epochs=EPOCHS, verbose=1, callbacks=[logger])

In [18]:
model.save('model_xception0819.h5')

In [19]:
from sklearn.preprocessing import LabelBinarizer
import pickle

lb = LabelBinarizer()
labels_copy = lb.fit_transform(labels_copy)
with open('lb_xception0819.pickle', 'wb') as f:
    f.write(pickle.dumps(lb))


In [22]:
plt.style.use('ggplot')
plt.figure()
N = EPOCHS
plt.plot(np.arange(0,N), H.history['loss'], label = 'train_loss')
plt.plot(np.arange(0,N), H.history['val_loss'], label = 'val_loss')
plt.plot(np.arange(0,N), H.history['accuracy'], label = 'train_acc')
plt.plot(np.arange(0,N), H.history['val_accuracy'], label = 'val_acc')
plt.title('Training Loss and Accuracy on car-type classifier')
plt.xlabel('Epoch #')
plt.ylabel('Loss/Accuracy')
plt.legend(loc='lower left')
plt.savefig('plot.png')

In [19]:
del model